In [192]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/train.csv
/kaggle/input/test.csv


In [193]:
dftest=pd.read_csv('/kaggle/input/test.csv')
dftrain=pd.read_csv('/kaggle/input/train.csv')
dftraintext=pd.read_csv('/kaggle/input/train.csv',usecols=['review_by_patient'])

In [194]:
dftrain['review_by_patient']=dftrain['review_by_patient']+" "+dftrain['use_case_for_drug']+" "+dftrain['name_of_drug']

In [195]:
import string
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text=text.replace(punctuation,'')
    return text

dftrain['review_by_patient']=dftrain['review_by_patient'].apply(remove_punctuations)

In [196]:
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
stop=set(stopwords.words('english'))
lemmatizer=WordNetLemmatizer()
def stop_stem_train(text):
    final_text=[]
    for i in text.split():
        if(i.strip().lower() not in stop):
            word=lemmatizer.lemmatize(i.strip())
            final_text.append(word)
    return " ".join(final_text)


dftrain['review_by_patient']=dftrain['review_by_patient'].apply(stop_stem_train)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [197]:
from nltk.tokenize import sent_tokenize
sentences=dftrain['review_by_patient'].apply(lambda x: nltk.sent_tokenize(x))

In [198]:
sentences.shape

(32165,)

In [199]:
dftrain=dftrain.drop(['patient_id','name_of_drug','use_case_for_drug','review_by_patient','drug_approved_by_UIC','review_by_patient'],axis=1)

In [200]:
dftrain=pd.concat([dftrain,sentences],axis=1)

In [268]:
#dftrain.head()

In [202]:
dftrain1=dftrain

In [203]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [204]:
dftrain['review_by_patient'].shape

(32165,)

In [205]:

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
def analyze_sentiment(dftrain):
    sentiments = []
    sid = SentimentIntensityAnalyzer()
    for i in range(dftrain.shape[0]):
        line = dftrain['review_by_patient'].iloc[i]
        sentiment = sid.polarity_scores(line)
        sentiments.append([sentiment['neg'], sentiment['pos'],sentiment['neu'], sentiment['compound']])
        dftrain[['neg', 'pos', 'neu', 'compound']] = pd.DataFrame(sentiments)
    return dftrain

analyze_sentiment(dftrain)

,effectiveness_rating,number_of_times_prescribed,base_score,review_by_patient,neg,pos,neu,compound
0,9,27,8.022969,[side effect take combination Bystolic 5 Mg Fi...,0.189,0.000,0.811,-0.4215
1,8,192,7.858458,[son halfway fourth week Intuniv became concer...,0.034,0.207,0.760,0.9169
2,5,17,6.341969,[used take another oral contraceptive 21 pill ...,0.060,0.122,0.818,0.7506
3,9,37,6.590176,[Suboxone completely turned life around feel h...,0.101,0.274,0.625,0.9344
4,2,43,6.144782,[2nd day 5mg started work rock hard erection h...,0.211,0.146,0.643,-0.5267
...,...,...,...,...,...,...,...,...
32160,9,89,6.963020,[taking Cymbalta 15 month First 30mg six month...,0.310,0.172,0.517,-0.8934
32161,6,0,0.899076,[Nexplanon since Dec 27 2016 got first period ...,0.000,0.094,0.906,0.7184
32162,9,25,6.241812,[panic attack social anxiety starting adolesce...,0.385,0.161,0.455,-0.9753
32163,8,22,7.940428,[Prozac 4 week nowafter starting 2 year ago te...,0.155,0.223,0.621,0.7820


In [206]:
dftrain.shape[0]

32165

In [207]:
dftrain = dftrain.drop('review_by_patient',axis=1)

In [269]:
#dftrain

In [209]:

my_list = []
def calculate_score(dftrain):
    for i in range(dftrain.shape[0]):
        neg = dftrain['neg'].iloc[i]
        pos = dftrain['pos'].iloc[i]
        neu = dftrain['neu'].iloc[i]
        if ((neg > pos) and (neg > neu)):
            my_list.append('0') 
        elif ((pos > neg) and (pos > neu)):
            my_list.append('1') 
        else:
            my_list.append('2') 
        
calculate_score(dftrain)       

In [211]:
dftrain['result'] = my_list

In [270]:
#dftrain

In [213]:
dftrain.dtypes

effectiveness_rating            int64
number_of_times_prescribed      int64
base_score                    float64
neg                           float64
pos                           float64
neu                           float64
compound                      float64
result                         object
dtype: object

In [214]:
dftrain['result'] = dftrain['result'].astype(int)

In [271]:
#dftrain

In [217]:
dftrain = dftrain.drop(['neg','pos','neu','compound'],axis=1)

In [272]:
#dftrain

In [219]:
dftrain

,effectiveness_rating,number_of_times_prescribed,base_score,result
0,9,27,8.022969,2
1,8,192,7.858458,2
2,5,17,6.341969,2
3,9,37,6.590176,2
4,2,43,6.144782,2
...,...,...,...,...
32160,9,89,6.963020,2
32161,6,0,0.899076,2
32162,9,25,6.241812,2
32163,8,22,7.940428,2


In [220]:
X = dftrain.drop('base_score',axis=1)

In [221]:

dftrain

,effectiveness_rating,number_of_times_prescribed,base_score,result
0,9,27,8.022969,2
1,8,192,7.858458,2
2,5,17,6.341969,2
3,9,37,6.590176,2
4,2,43,6.144782,2
...,...,...,...,...
32160,9,89,6.963020,2
32161,6,0,0.899076,2
32162,9,25,6.241812,2
32163,8,22,7.940428,2


In [222]:
Y = dftrain.drop(['effectiveness_rating','number_of_times_prescribed','result'],axis=1)
Y = Y.astype(int)
Y

,base_score
0,8
1,7
2,6
3,6
4,6
...,...
32160,6
32161,0
32162,6
32163,7


In [223]:
#Y = dftrain.drop(['effectiveness_rating','number_of_times_prescribed','result'],axis=1)

In [224]:
#X = dftrain.drop('base_score',axis=1)
X

,effectiveness_rating,number_of_times_prescribed,result
0,9,27,2
1,8,192,2
2,5,17,2
3,9,37,2
4,2,43,2
...,...,...,...
32160,9,89,2
32161,6,0,2
32162,9,25,2
32163,8,22,2


In [225]:
Y

,base_score
0,8
1,7
2,6
3,6
4,6
...,...
32160,6
32161,0
32162,6
32163,7


In [226]:
#dftrain["Base_score"] = pd.to_numeric(dftrain["base_score"],errors='coerce')

In [227]:
#dftrain = dftrain.drop('base_score',axis=1)
dftrain.dtypes

effectiveness_rating            int64
number_of_times_prescribed      int64
base_score                    float64
result                          int64
dtype: object

In [228]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
#y=Y['base_score'].values
#y = to_categorical(y)

In [229]:
Y.dtypes

base_score    int64
dtype: object

In [230]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.2)

In [231]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression


In [232]:
clf = DecisionTreeClassifier()
regressor = RandomForestRegressor()
regressor1 = LinearRegression()  
clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [233]:
y_pred = clf.predict(X_test)
y_pred

array([6, 6, 7, ..., 6, 8, 7])

In [234]:
#clf1= clf.fit(X_train,y_train)
clf2 = regressor.fit(X_train,y_train)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [235]:
#y_pred = clf2.predict(X_test)
#y_pred
np.around(y_pred, decimals=0)

array([6, 6, 7, ..., 6, 8, 7])

In [236]:
from sklearn import metrics 
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
Accuracy: 0.9900512979947148

Accuracy: 0.9900512979947148
